In [1]:
import pandas as pd
import numpy as np
import torch
import os

In [2]:
# Renaming
# tic_list = [tic[:-4] for tic in os.listdir("./CSI")]
# for i in range(len(tic_list)):
#     data = pd.read_csv(f"./CSI/{tic_list[i]}.csv", index_col=0)
#     os.remove(f"./CSI/{tic_list[i]}.csv")
#     if tic_list[i][-2:] == "SS":
#         mark = "SH"
#     else:
#         mark = "SZ"
#     tic_list[i] = mark + tic_list[i][:6]
#     data.to_csv(f"./CSI/{tic_list[i]}.csv")

In [9]:
import qlib
from qlib.config import REG_CN
from qlib.contrib.data.handler import Alpha158

qlib.init(provider_uri="~/.qlib/CSI_data", region=REG_CN)
data_handler_config = {
    "start_time": "2018-01-01",
    "end_time": "2022-05-01",
    "instruments": "all",
}

[12664:MainThread](2024-02-29 11:19:50,460) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[12664:MainThread](2024-02-29 11:19:50,464) WARNING - qlib.Initialization - [__init__.py:64] - auto_path is False, please make sure None is mounted
[12664:MainThread](2024-02-29 11:19:50,467) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[12664:MainThread](2024-02-29 11:19:50,469) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/胡逸凡/.qlib/qlib_data/cn_data')}


In [10]:
h = Alpha158(**data_handler_config)

ValueError: instrument: {'__DEFAULT_FREQ': 'C:\\Users\\胡逸凡\\.qlib\\qlib_data\\cn_data'} does not contain data for day

In [ ]:
data_df = h.fetch()

In [ ]:
feature_df = data_df.reset_index().dropna(axis=1, how="all").rename(
    columns={"datetime": "date", "instrument": "tic"}
)
feature_df


In [ ]:
# Strange features that have different NaN data among different stocks.

drop_col = []
for k,v in feature_df.isna().sum().items():
    if v % 100 != 0:
        print(k, v)
        drop_col.append(k)
            

In [ ]:
feature_df = feature_df.drop(columns=drop_col)

In [ ]:
feature_df = feature_df.dropna()

In [ ]:
for tic in feature_df['tic'].unique():
    print(len(feature_df[feature_df['tic'] == tic]))

In [18]:
# alpha158 = feature_df.columns[2:].to_list() # alpha158
basic_feature = ["open", "close", "high", "low", "volume"]

In [19]:
target_return_span = 5
target = f"return+{target_return_span}"

In [20]:
time_span = 60

CSI_date = ['20110419', '20181228', '20180102', '20201231',  '20190402', '20211231']

In [21]:
tic_df_list = []
for tic_path in os.listdir("./CSI/"):
    tic_df = pd.read_csv(f"./CSI/{tic_path}", index_col=0)[["date", "open", "close", "high", "low", "volume"]]
    tic_df["tic"] = tic_path[:8]
    tic_df[target] = tic_df.close.pct_change(target_return_span).shift(-1 * target_return_span)
    tic_df_list.append(tic_df)

tic_target_df = pd.concat(tic_df_list).sort_values(by="date").dropna()
tic_target_df.date = pd.DatetimeIndex(tic_target_df.date)


In [22]:
tic_target_df

,date,open,close,high,low,volume,tic,return+5
0,2010-01-04,9.032978,5.803878,9.049530,8.755741,159964606.0,SH600000,-0.023596
0,2010-01-04,16.675959,10.920465,16.843369,16.424843,80224538.0,SH600036,-0.043478
0,2010-01-04,17.716665,11.288666,18.693333,17.716665,11110185.0,SZ000895,-0.047533
0,2010-01-04,31.799999,25.086197,32.000000,31.360001,26409039.0,SZ000858,-0.056916
0,2010-01-04,7.370151,4.855292,7.422748,7.192636,84411995.0,SH600048,-0.041096
...,...,...,...,...,...,...,...,...
2989,2022-04-26,22.920000,23.190001,23.389999,22.860001,43810129.0,SH600900,-0.020267
2989,2022-04-26,6.880000,6.820000,6.990000,6.800000,46892044.0,SH601006,-0.046921
2989,2022-04-26,34.200001,35.709999,36.500000,33.980000,74157971.0,SZ300015,0.013162
2989,2022-04-26,32.349998,32.880001,34.049999,31.900000,23586836.0,SH600660,0.095195


In [23]:
# dataset_df = feature_df.merge(tic_target_df, how='inner', on=["date", "tic"])
dataset_df = tic_target_df
dataset_df

,date,open,close,high,low,volume,tic,return+5
0,2010-01-04,9.032978,5.803878,9.049530,8.755741,159964606.0,SH600000,-0.023596
0,2010-01-04,16.675959,10.920465,16.843369,16.424843,80224538.0,SH600036,-0.043478
0,2010-01-04,17.716665,11.288666,18.693333,17.716665,11110185.0,SZ000895,-0.047533
0,2010-01-04,31.799999,25.086197,32.000000,31.360001,26409039.0,SZ000858,-0.056916
0,2010-01-04,7.370151,4.855292,7.422748,7.192636,84411995.0,SH600048,-0.041096
...,...,...,...,...,...,...,...,...
2989,2022-04-26,22.920000,23.190001,23.389999,22.860001,43810129.0,SH600900,-0.020267
2989,2022-04-26,6.880000,6.820000,6.990000,6.800000,46892044.0,SH601006,-0.046921
2989,2022-04-26,34.200001,35.709999,36.500000,33.980000,74157971.0,SZ300015,0.013162
2989,2022-04-26,32.349998,32.880001,34.049999,31.900000,23586836.0,SH600660,0.095195


In [24]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 299000 entries, 0 to 2989
Data columns (total 8 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   date      299000 non-null  datetime64[ns]
 1   open      299000 non-null  float64       
 2   close     299000 non-null  float64       
 3   high      299000 non-null  float64       
 4   low       299000 non-null  float64       
 5   volume    299000 non-null  float64       
 6   tic       299000 non-null  object        
 7   return+5  299000 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 20.5+ MB


In [35]:
dataset_df['date'].nunique()

2990

In [25]:
train_df = dataset_df[(dataset_df.date >= CSI_date[0]) & (dataset_df.date <= CSI_date[1])]
val_df = dataset_df[(dataset_df.date >= CSI_date[2]) & (dataset_df.date <= CSI_date[3])]
test_df = dataset_df[(dataset_df.date >= CSI_date[4]) & (dataset_df.date <= CSI_date[5])]

In [38]:
print(test_df['date'].nunique(), ' ', train_df['date'].nunique(), ' ', val_df['date'].nunique())

669   1877   728


In [26]:
# tic_list = dataset_df.tic.unique()

# with open("tic_list.txt", "w") as f:
#     for tic in tic_list:
#         f.write(tic + "\n")

In [27]:
with open("tic_list.txt", "r") as f:
    tic_list = [tic.strip("\n") for tic in f.readlines()]

In [28]:
import torch

In [31]:
def df_2_array(dataset_df, feat_col, target, type):
    dataset_feat = []
    dataset_ret = []
    dataset_price = []
    for tic in tic_list:
        df = dataset_df[dataset_df.tic == tic]
        feat = df[feat_col].to_numpy()
        ret = df[target].to_numpy()
        price = df['close'].to_numpy()
        stock_feat = []
        stock_ret = []
        stock_price = []
        for i in range(time_span, feat.shape[0]):
            stock_feat.append(feat[i-time_span : i])
            stock_ret.append(ret[i])
            stock_price.append(price[i])
        stock_feat = np.array(stock_feat)
        stock_ret = np.array(stock_ret)
        stock_price = np.array(stock_price)
    
        dataset_feat.append(stock_feat)
        dataset_ret.append(stock_ret)
        dataset_price.append(stock_price)

    dataset_feat = np.array(dataset_feat).transpose((1, 2, 0, 3))
    dataset_ret = np.array(dataset_ret).transpose((1, 0))
    dataset_price = np.array(dataset_price).transpose((1, 0))
    
    dataset_feat_tensor = torch.tensor(dataset_feat, dtype=torch.float)
    dataset_ret_tensor = torch.tensor(dataset_ret, dtype=torch.float)
    dataset_price_tensor = torch.tensor(dataset_price, dtype=torch.float)
    
    torch.save(dataset_feat_tensor, f"./dataset/{type}/feat.pt")
    torch.save(dataset_ret_tensor, f"./dataset/{type}/ret.pt")
    torch.save(dataset_price_tensor, f"./dataset/{type}/price.pt")    
    
    return dataset_feat, dataset_ret, dataset_price

In [32]:
_ = df_2_array(train_df, basic_feature, target, "train")
_ = df_2_array(val_df, basic_feature, target, "val")
dataset_feat, dataset_ret, dataset_price = df_2_array(test_df, basic_feature, target, "test")

In [33]:
dataset_feat.shape

(609, 60, 100, 5)

In [34]:
dataset_ret.shape

(609, 100)